In [77]:
import pandas as pd
import numpy as np

from scipy.stats import mannwhitneyu
from scipy import stats

In [103]:
df_titanic = pd.read_csv("./data/titanic.csv")
df_housing = pd.read_csv("./data/ejemplo_housing.csv")
df_car = pd.read_csv("./data/car_price.csv")

### Funcion: tipifica_variables

Esta función debe recibir como argumento un dataframe, un entero (`umbral_categoria`) y un float (`umbral_continua`). La función debe devolver un dataframe con dos columnas "nombre_variable", "tipo_sugerido" que tendrá tantas filas como columnas el dataframe. En cada fila irá el nombre de una de las columnas y una sugerencia del tipo de variable. Esta sugerencia se hará siguiendo las siguientes pautas:
+ Si la cardinalidad es 2, asignara "Binaria"
+ Si la cardinalidad es menor que `umbral_categoria` asignara "Categórica"
+ Si la cardinalidad es mayor o igual que `umbral_categoria`, entonces entra en juego el tercer argumento:
    * Si además el porcentaje de cardinalidad es superior o igual a `umbral_continua`, asigna "Numerica Continua"
    * En caso contrario, asigna "Numerica Discreta"


In [79]:
def tipifica_variables(dataframe, umbral_categoria = int, umbral_continua = float):

    df_resultado = pd.DataFrame([dataframe.nunique(), dataframe.nunique()/len(dataframe)*100]).T.rename(columns = {0: "Cardinalidad", 1: "%_Cardinalidad"})
    
    df_resultado["Tipo"] = "Numérica Discreta"
    df_resultado.loc[df_resultado["Cardinalidad"] < umbral_categoria, "Tipo"] = "Categórica"
    df_resultado.loc[df_resultado["Cardinalidad"] == 2.0, "Tipo"] = "Binaria"
    df_resultado.loc[df_resultado["%_Cardinalidad"] >= umbral_continua, "Tipo"] = "Numérica Continua"

    return df_resultado

In [80]:
# prueba

tipifica_variables(df_titanic, 5, 25)

,Cardinalidad,%_Cardinalidad,Tipo
sex,2.0,0.224467,Binaria
age,89.0,9.988777,Numérica Discreta
sibsp,7.0,0.785634,Numérica Discreta
parch,7.0,0.785634,Numérica Discreta
fare,248.0,27.833895,Numérica Continua
class,3.0,0.336700,Categórica
who,3.0,0.336700,Categórica
adult_male,2.0,0.224467,Binaria
embark_town,3.0,0.336700,Categórica
alive,2.0,0.224467,Binaria


### Funcion: get_features_cat_regression

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad y una variable float "pvalue" cuyo valor por defecto será 0.05.

La función debe devolver una lista con las columnas categóricas del dataframe cuyo test de relación con la columna designada por 'target_col' supere en confianza estadística el test de relación que sea necesario hacer (es decir la función debe poder escoger cuál de los dos test que hemos aprendido tiene que hacer).

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.


[PENDING] Try and except ---> evaluar los valores asignados a los argumentos de entrada y retornar 'None' con el error correspondiente.

[CHECK!!] Llamar a la función cardinalidad para que nos tipifique las columnas del dataset y guardar las categóricas en una variable

[CHECK!!] Poner inputs para que el usuario establezca los umbrales requeridos en los argumentos de la cardinalidad

[CHECK] Análisis un bivariante de las categóricas contra la target, medir ratios y frecuencias 

[CHECK] Análisis bivariante numérica (target) vs categórica (lista de categóricas) mediante test de hipótesis U de Mann-Whitney & ANOVA

[CHECK] Que distinga cuál de los dos tests debe aplicar: U de Mann-Whitney --> Binarias   ///  ANOVA --> resto de categóricas

[CHECK] Return --> lista de variables que superen en confianza estadística el test de relación pertinente

In [88]:
def get_features_cat_regression(dataframe, target_col = float, pvalue = 0.05): 
    
    """
    DESCRIPCIÓN:

    Análisis bivariante de la variable target contra las variables categóricas de un dataset para su posterior selección de features categóricas ante la construcción de un 
    hipotético modelo de regresión lineal.

    REQUISITOS:
    
    Es necesario importar mannwhitneyu y stats de scipy, ejecuta:
    
    from scipy.stats import mannwhitneyu 
    
    from scipy import stats

    ARGUMENTOS:

    param1 (DataFrame): Dataset de train del conjunto de datos de un hipotético modelo de regresión lineal que queremos entrenar

    param2 (float): Columna del dataset que constiuye el 'target' de nuestro hipotético modelo de regresión. Variable numérica continua o discreta con alta cardinalidad

    param3 (float): Valor del pvalue (default = 0.05) 

    RETURN:

    (list): Variables categóricas que superen en confianza estadística el test de relación pertinente tras un análisis bivariante.

    """
    input_cat = int(input("CÁLCULO CARDINALIDAD DE VARIABLES. Introduce un valor para el umbral de categorias (en cifras, ej: 3; no escribas 'tres')"))
    input_num = int(input("CÁLCULO CARDINALIDAD DE VARIABLES. Introduce un valor para el umbral de las númericas continuas (en cifras, ej: 30; no escribas 'treinta')"))
    
    # tipificamos en función de la cardinalidad y lo instanciamos
    df_tipo = tipifica_variables(dataframe, umbral_categoria= input_cat, umbral_continua= input_num)   

    # me quedo con las categóricas y las vuelco en una lista el nombre de la variable, que está en el índice del dataset
    
    es_catego = df_tipo.Tipo == "Categórica"
    es_binaria = df_tipo.Tipo == "Binaria"

    lista_categoricas = df_tipo.loc[es_catego | es_binaria].index.to_list()

    for categoria in lista_categoricas:
        features_categoricas = []

        # si mi variable es binaria, aplicamos U de Mann-Whitney

        if len(dataframe[categoria].unique()) == 2:      
            
            # from scipy.stats import mannwhitneyu --> preguntar si esto debería ir aquí o añadir en el stringdoc que es necesario importarlo para usar la función
            
            es_a = dataframe[categoria].unique()[0]   # obtengo las dos agrupaciones
            es_b = dataframe[categoria].unique()[1]
            
            grupo_a = dataframe.loc[dataframe[categoria] == es_a][target_col]   # y separo mi dataset en función de ellas
            grupo_b = dataframe.loc[dataframe[categoria] == es_b][target_col]
            
            u_stat, p_valor = mannwhitneyu(grupo_a, grupo_b)

            if p_valor <= 0.05:
                features_categoricas.append(categoria)
            

        # si no es binaria, aplicamos ANOVA

        else:   
            # from scipy import stats 
            grupos = dataframe[categoria].unique()  # obtengo todos valores de la variable

            # obtenemos los valores del target por cada valor de las diferentes categorias con un list comprehension 
            argumento_stats = [dataframe[dataframe[categoria] == grupo][target_col] for grupo in grupos] 
                 
            f_val, p_valor = stats.f_oneway(*argumento_stats) # El método * separa todos los elementos de la lista y los pasa como argumento a la función                                                   

            if p_valor <= 0.05:
                features_categoricas.append(categoria)

    return features_categoricas  


In [ ]:
get_features_cat_regression(df_titanic, "fare")

['alone']

In [89]:
get_features_cat_regression(df_housing, "median_house_value")

['ocean_proximity']

In [105]:
get_features_cat_regression(df_car, "price")

['enginelocation']

In [104]:
df_car.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')